# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [55]:
# import libraries
import sys
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sqlalchemy import create_engine

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# nlp libraries
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download(['stopwords', 'punkt', 'wordnet'])

# sklearn libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\heuse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\heuse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\heuse\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
# load data from database
engine = create_engine('sqlite:///data/DisasterData.db')
df = pd.read_sql_table(table_name='tbl_disastermessages', con=engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [27]:
text = X[1]
print(text)

# Convert to lowercase
text = text.lower()
print(text)
# Remove punctuation characters
text = re.sub(r"[^a-zA-Z0-9]", " ", text)
print(text)

#Split text into words using NLTK
words = word_tokenize(text)
print(words)

# Combine in one
words = word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
print(words)

[t for t in words if t not in stopwords.words('english')]


Is the Hurricane over or is it not over
is the hurricane over or is it not over
is the hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']


['hurricane']

In [29]:
def tokenize(text):
    """Tokenzize input text into words and returns the base from the word based on english.

    Args:
        text (str): input message

    Returns:
        tokens (list of str): A returning list of tokens found in the input message
    """

    # Normalize and tokenize and remove punctuation.
    words = word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))

    # Remove stopwords
    tokens = [t for t in words if t not in stopwords.words('english')]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [30]:
#Random Forest classifier
pipeline_rf = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [31]:
# Splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Fit the model 
pipeline_rf.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002DAEF8B8310>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
# f1 score, precision and recall with classifiation_report
def plot_scores(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    for i, column in enumerate(y_test):
        print('Feature {}: {}'.format(i+1, column))
        print(classification_report(y_test[column], y_pred[:, i]))
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [33]:
# Prediction the model  
y_pred = pipeline_rf.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.72      0.42      0.54      1575
           1       0.83      0.95      0.89      4927
           2       0.47      0.38      0.42        52

    accuracy                           0.82      6554
   macro avg       0.67      0.59      0.61      6554
weighted avg       0.81      0.82      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5446
           1       0.84      0.51      0.64      1108

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       

c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6390
           1       1.00      0.02      0.04       164

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.96      6554

Feature 16: missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6492
           1       1.00      0.02      0.03        62

    accuracy                           0.99      6554
   macro avg       1.00      0.51      0.51      6554
weighted avg       0.99      0.99      0.99      6554

Feature 17: refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6346
           1       0.75      0.03      0.06       208

    accuracy                           0.97      6554
   macro avg       0.86      0.51      0.52      6554
weighted avg       0.96   

c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6487
           1       0.00      0.00      0.00        67

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Feature 26: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Feature 27: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6482
           1       0.00      0.00      0.00        72

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
pipeline_rf.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000002DAEF8B8310>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000002DAEF8B8310>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [35]:
parameters_rf = {
    #'tfidf__use_idf': [True, False],    
    'clf__estimator__n_estimators': [5,10],
    'clf__estimator__min_samples_split':[4,6]
}

cv_rf = GridSearchCV(pipeline_rf, param_grid = parameters_rf, verbose=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
# Fit the model again with the grid search parameters
cv_rf.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5;, score=0.209 total time= 2.4min
[CV 2/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5;, score=0.212 total time= 2.2min
[CV 3/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5;, score=0.206 total time= 2.2min
[CV 4/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5;, score=0.204 total time= 2.1min
[CV 5/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5;, score=0.213 total time= 2.1min
[CV 1/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10;, score=0.248 total time= 2.2min
[CV 2/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10;, score=0.230 total time= 2.1min
[CV 3/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10;, score=0.241 total time= 2.3min
[CV 4/5] END clf__estimat

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002DAEF8B8310>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [4, 6],
                         'clf__estimator__n_estimators': [5, 10]},
             verbose=4)

In [37]:
# Evaulate the model again with grid search
y_pred = cv_rf.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.66      0.41      0.51      1575
           1       0.83      0.93      0.88      4927
           2       0.68      0.37      0.47        52

    accuracy                           0.80      6554
   macro avg       0.72      0.57      0.62      6554
weighted avg       0.79      0.80      0.79      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      5446
           1       0.78      0.48      0.60      1108

    accuracy                           0.89      6554
   macro avg       0.84      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       

c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5822
           1       0.83      0.49      0.61       732

    accuracy                           0.93      6554
   macro avg       0.88      0.74      0.79      6554
weighted avg       0.93      0.93      0.92      6554

Feature 13: shelter
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5996
           1       0.79      0.39      0.52       558

    accuracy                           0.94      6554
   macro avg       0.87      0.69      0.74      6554
weighted avg       0.93      0.94      0.93      6554

Feature 14: clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6448
           1       0.74      0.24      0.36       106

    accuracy                           0.99      6554
   macro avg       0.86      0.62      0.68      6554
weighted avg       0.98      0.99

c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heuse\anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      5973
           1       0.73      0.53      0.61       581

    accuracy                           0.94      6554
   macro avg       0.84      0.76      0.79      6554
weighted avg       0.94      0.94      0.94      6554

Feature 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       1.00      0.01      0.03        71

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

Feature 33: earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5924
           1       0.89      0.77      0.82       630

    accuracy                           0.97      6554
   macro avg       0.93      0.88      0.90      6554
weighted avg       0.97      0.97 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [57]:
# Export a pickle file for the final model
cwd = os.getcwd()
filename = '\data\classifier.pkl'

with open (cwd+filename, 'wb') as f:
    pickle.dump(cv_rf, f)

PicklingError: Can't pickle <function tokenize at 0x000002DAEF8B8310>: it's not the same object as __main__.tokenize

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [39]:
# import libraries


def load_data(db_file_path):
    engine = create_engine('sqlite://db_file_path')
    df = pd.read_sql_table(table_name='tbl_disastermessages', con=engine)
    X = df['message']
    Y = df.iloc[:,4:]
    return X,Y

def tokenize(text):
    """Tokenzize input text into words and returns the base from the word based on english.

    Args:
        text (str): input message

    Returns:
        tokens (list of str): A returning list of tokens found in the input message
    """

    # Normalize and tokenize and remove punctuation.
    words = word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))

    # Remove stopwords
    tokens = [t for t in words if t not in stopwords.words('english')]
    #Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens]

    return tokens


def build_model():
    pass


def train_model(X,y, model):
    # trains test split
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    # train model
    model.fit(X_train, y_train)
    # predict model
    y_pred = model.predict(X_test)
    return model


def export_model(model):
    # Export a pickle file for the final model
    filename = 'classifier.pkl'
    with open (filename, 'wb') as f:
        pickle.dump(cv_rf, f)
